<a href="https://colab.research.google.com/github/NhutTien0905/NhutTien0905/blob/main/flower_classification_2_hidden.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/file/iris.csv")
data.head()

,senpal lenght,senpal width,pental length,pental width,type
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
data = data.sample(frac = True)
data

,senpal lenght,senpal width,pental length,pental width,type
44,5.1,3.8,1.9,0.4,Iris-setosa
122,7.7,2.8,6.7,2.0,Iris-virginica
94,5.6,2.7,4.2,1.3,Iris-versicolor
52,6.9,3.1,4.9,1.5,Iris-versicolor
128,6.4,2.8,5.6,2.1,Iris-virginica
...,...,...,...,...,...
86,6.7,3.1,4.7,1.5,Iris-versicolor
138,6.0,3.0,4.8,1.8,Iris-virginica
12,4.8,3.0,1.4,0.1,Iris-setosa
106,4.9,2.5,4.5,1.7,Iris-virginica


In [4]:
X = data.drop(columns = "type").to_numpy()
Y = data[["type"]].to_numpy()
print(X[0].shape)
print(Y[0].shape)

(4,)
(1,)


In [5]:
def TypetoVec(n,maxlength = 3):
    vec = np.zeros(maxlength)
    if n == "Iris-setosa":
        vec[0] = 1
    elif n == "Iris-versicolor":
        vec[1] = 1
    else:
        vec[2] = 1
    return vec.reshape(1,maxlength)

In [6]:
trainX = []
trainY = []
trainPair = list(zip(X,Y))
random.shuffle(trainPair)
for i in range(len(X)):
    x,y = trainPair[i]
    #x = x.flatten()
    x = x.reshape(1,len(x))
    trainX.append(x)
    y = TypetoVec(y)
    trainY.append(y)

print(trainX[0].shape)
print(trainY[0].shape)

(1, 4)
(1, 3)


In [45]:
class NeuralNetwork():
    def __init__(self,Xs,Ys,inputS,hiddenS1,hiddenS2,outputS):
        self.Xs = Xs
        self.Ys = Ys
        self.inputS = inputS
        self.hiddenS1 = hiddenS1
        self.hiddenS2 = hiddenS2
        self.outputS = outputS

    def generateWeight(self,inputt,outputt):
        return np.random.rand(outputt,inputt + 1).transpose() - 0.5

    def sigmoid(self,z):
        return 1/(1+np.exp(-z))

    def addBias(self,x):
        curLen = len(x.flatten())
        return np.concatenate(([1],x.flatten())).reshape(1,curLen+1)

    def forward(self,x,w1,w2,w3):
        a1 = x #(1,784)
        a1_bias = self.addBias(a1) #(1,785)
        z2 = np.matmul(a1_bias,w1) #(1,785)x(785,100)
        a2 = self.sigmoid(z2) #(1,100)
        a2_bias = self.addBias(a2) #(1,101)
        z3 = np.matmul(a2_bias,w2) #(1,101)x(101,100)
        a3 = self.sigmoid(z3) # (1,100)
        a3_bias = self.addBias(a3) #(1,101)
        z4 = np.matmul(a3_bias,w3) #(1,101)x(101,10)
        a4 = self.sigmoid(z4) #(1,10)
        return a1_bias,a2,a2_bias,a3,a3_bias,a4

    def backward(self,x,y,w1,w2,w3):
        a1_bias,a2,a2_bias,a3,a3_bias,a4 = self.forward(x,w1,w2,w3)

        d4 = a4 - y #(1,10)
        # (1,10)x(10,101) * (1,101)
        d3 = (np.matmul(d4,w3.transpose())) * (a3_bias*(1-a3_bias))
        # (1,100)x(100,101) * (1,101)
        d2 = (np.matmul(d3[:,1:],w2.transpose())) * (a2_bias*(1-a2_bias))

        delta1 = np.matmul(a1_bias.transpose(), d2) # (785,1)x(1,101)->(785,101)
        delta2 = np.matmul(a2_bias.transpose(), d3) # (101,1)x(1,101)->(101,101)
        delta3 = np.matmul(a3_bias.transpose(), d4) # (101,1)x(1,10)->(101,10)

        return delta1,delta2,delta3

    def mse(self,pred,y):
        return np.mean(np.power(pred - y,2))

    def predict(self,xtest,w1,w2,w3):
        a1_bias,a2,a2_bias,a3,a3_bias,a4 = self.forward(xtest,w1,w2,w3)
        # a4[a4 > 0.5] = 1
        # a4[a4 <= 0.5] = 0
        return a4

    def train(self,epochs,w1,w2,w3,lr):
        acc = []
        losses = []
        for epoch in range(epochs):
            capDelta1 = 0
            capDelta2 = 0
            capDelta3 = 0
            correctCase = 0
            l = []
            for i in range(len(self.Xs)):
                x = self.Xs[i]
                y = self.Ys[i]
                a1_bias,a2,a2_bias,a3,a3_bias,pred = self.forward(x,w1,w2,w3)
                if pred.argmax() == y.argmax():
                    correctCase += 1
                l.append(self.mse(pred,y))
                delta1,delta2,delta3 = self.backward(x,y,w1,w2,w3)
                capDelta1 += delta1
                capDelta2 += delta2
                capDelta3 += delta3
            # (785,100)     (785,100)
            w1 = w1 - lr*capDelta1[:,1:]
            # (101,100)     (101,100)
            w2 = w2 - lr*capDelta2[:,1:]
            # (101,10)      (101,10)
            w3 = w3 - lr*capDelta3

            curAcc = correctCase*100/len(self.Xs)
            acc.append(curAcc)
            meanLoss = np.array(l).mean()
            losses.append(meanLoss)
            freq = 5
            if (epoch + 1) % freq == 0:
                avgAcc = np.array(acc[-5:]).mean()
                loss = np.array(losses[-5:]).mean()
                print(f"Epoch {epoch + 1}: AVG ACC during {freq} epochs: {avgAcc}")
                print(f"Epoch {epoch + 1}: Loss: {loss}")
        return acc,losses,w1,w2,w3

        

In [8]:
inputS = 4
hiddenS1 = 25
hiddenS2 = 25
outputS = 3

In [46]:
neural = NeuralNetwork(trainX,trainY,inputS,hiddenS1,hiddenS2,outputS)
w1 = neural.generateWeight(inputS,hiddenS1) # (100,785) -> (785,100)
w2 = neural.generateWeight(hiddenS1,hiddenS2) # (100,101) -> (101,100)
w3 = neural.generateWeight(hiddenS2,outputS) # (10,101) -> (101,10)

In [57]:
lr = 0.001
acc,losses,w1,w2,w3 = neural.train(50,w1,w2,w3,lr)

Epoch 5: AVG ACC during 5 epochs: 98.0
Epoch 5: Loss: 0.04609607117824038
Epoch 10: AVG ACC during 5 epochs: 98.0
Epoch 10: Loss: 0.044990347398949256
Epoch 15: AVG ACC during 5 epochs: 98.0
Epoch 15: Loss: 0.043902872916630795
Epoch 20: AVG ACC during 5 epochs: 98.0
Epoch 20: Loss: 0.04283494249310482
Epoch 25: AVG ACC during 5 epochs: 98.0
Epoch 25: Loss: 0.041787737389228544
Epoch 30: AVG ACC during 5 epochs: 98.0
Epoch 30: Loss: 0.04076231924141388
Epoch 35: AVG ACC during 5 epochs: 98.0
Epoch 35: Loss: 0.03975962584563118
Epoch 40: AVG ACC during 5 epochs: 98.0
Epoch 40: Loss: 0.03878046880356035
Epoch 45: AVG ACC during 5 epochs: 98.0
Epoch 45: Loss: 0.037825532942500975
Epoch 50: AVG ACC during 5 epochs: 98.0
Epoch 50: Loss: 0.03689537738469152


In [34]:
xtest = np.array([4.9,2.5,4.5,1.7])
pred = neural.predict(xtest,w1,w2,w3)
print(pred)

[[0.00144857 0.91547847 0.04744419]]


In [58]:
xtestt = np.array([6.8,2.8,4.8,1.4]) # type 2
pred = neural.predict(xtestt,w1,w2,w3)
print(pred)

[[0.01353374 0.66771449 0.1570226 ]]


In [59]:
xtestt1 = np.array([5.1,3.5,1.5,0.3]) # type 1
pred = neural.predict(xtestt1,w1,w2,w3)
print(pred)

[[9.75982420e-01 9.43051024e-02 9.18481967e-05]]


In [60]:
xtestt2 = np.array([7.8,2.8,5.8,2.4]) # type 3
pred = neural.predict(xtestt2,w1,w2,w3)
print(pred)

[[6.54793426e-04 3.06589366e-01 8.61487099e-01]]
